In [ ]:
##################################################################################################################
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.rcParams['image.cmap'] = 'gray'
##################################################################################################################

In [ ]:
# ##################################################################################################################
# ## get device configuration ...
# physical_devices = tf.config.experimental.list_physical_devices('GPU')

# ## print('    [!] number of usable devices: ', len(physical_devices))
# device_id = 0
# world_size = 1

# tf.config.experimental.set_memory_growth(physical_devices[device_id], True)
# tf.config.experimental.set_visible_devices(physical_devices[device_id], 'GPU')
# ##################################################################################################################

In [ ]:
##################################################################################################################
# example of pix2pix gan for satellite to map image-to-image translation
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
import tensorflow as tf
import keras
from keras.initializers import RandomNormal
from keras.layers import Input, Model
from keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization

# define the discriminator model
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model

# define an encoder block
def encoder_block(layer_in, n_filters, s, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(s,s), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g

# define a resnet block
def resnet_block(n_filters, input_layer, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(input_layer)
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	g = LeakyReLU(alpha=0.2)(g)
	# second convolutional layer
	g = Conv2D(n_filters, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(g)
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g

# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add upsampling layer
	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g

# define the standalone generator model
def define_generator(image_shape=(224,224)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape) 
	# encoder model
	e1 = encoder_block(in_image, 64, 2, batchnorm=False) #112

	e2 = encoder_block(e1, 2, 128) #56
	for _ in range(3):
		r2 = resnet_block(128, e2)  
	act2 = LeakyReLU(alpha=0.2)(r2) 
	
	e3 = encoder_block(act2, 2, 256) #28
	for _ in range(4):
		r3 = resnet_block(256, e3) 
	act3 = LeakyReLU(alpha=0.2)(r3) 

	e4 = encoder_block(act3, 2, 512) #14
	for _ in range(6):
		r4 = resnet_block(512, e4) 
	act4 = LeakyReLU(alpha=0.2)(r4) 

	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(act4) #7
	b = Activation('relu')(b)  

	# decoder model
	d4 = decoder_block(b, act4, 512) #14
	d3 = decoder_block(d4, act3, 256)  #28
	d2 = decoder_block(d3, act2, 128, dropout=False) #56
	d1 = decoder_block(d2, e1, 64, dropout=False) #112

	# output
	g = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d1)
	out_image = Activation('sigmoid')(g)
	# define model
	model = Model(in_image, out_image)
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# define the source image
	in_src = Input(shape=image_shape)
	# connect the source image to the generator input
	gen_out = g_model(in_src)
	# connect the source input and generator output to the discriminator input
	dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
	model = Model(in_src, [dis_out, gen_out])
	# compile model
	opt = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)  # 원래 베타값 0.5
	model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
	return model

In [ ]:
# load and prepare training images
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	return [X1, X2]

# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

# generate samples and save as a plot and save the model
def summarize_performance(step, d_model, g_model, gan_model, dataset, n_samples=3):
	# select a sample of input images
	[X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
	# generate a batch of fake samples
	X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
	# plot real source images
	for i in range(n_samples):
		plt.subplot(3, n_samples, 1 + i)
		plt.axis('off')
		plt.imshow(X_realA[i])   # input image
	# plot generated target image
	for i in range(n_samples):
		plt.subplot(3, n_samples, 1 + n_samples + i)
		plt.axis('off')
		plt.imshow(X_fakeB[i])   # predict image
	# plot real target image
	for i in range(n_samples):
		plt.subplot(3, n_samples, 1 + n_samples*2 + i)
		plt.axis('off')
		plt.imshow(X_realB[i])   # ground truth image
	# save plot to file
	filename1 = 'plot_%07d.png' % (step+1)
	plt.savefig(filename1)
	plt.close()
	# save the generator model
	filename2 = 'g_model_%07d.h5' % (step+1)
	g_model.save(filename2)
	filename3 = 'gan_model_%07d.h5' % (step+1)
	gan_model.save(filename3)
	filename4 = 'd_model_%07d.h5' % (step+1)
	d_model.save(filename4)
	print('>Saved: %s, %s and %s' % (filename1, filename2, filename3))
##################################################################################################################

In [ ]:
##################################################################################################################
import pickle
# train pix2pix models  # 27720 data files
def train(d_model, g_model, gan_model, dataset, n_epochs=50, n_batch=1):
	# determine the output square shape of the discriminator
	t = time.time()
	n_patch = d_model.output_shape[1]
	# unpack dataset
	trainA, _ = dataset
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)   # bat_per_epoch = 27720
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs    # 27720*15 = 415800 step
	# manually enumerate epochs
	D_loss, G_loss = {}, {}
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		# update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		d_loss = (d_loss1 + d_loss2) / 2
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		D_loss.setdefault(i, d_loss)
		G_loss.setdefault(i, g_loss)
		# summarize performance
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss), (time.time() - t)/60, " minute")
		# summarize model performance
		if (i+1) % (bat_per_epo) == 0:
			summarize_performance(i, d_model, g_model, gan_model, dataset)
			titles_ = ['Discriminator Loss', 'Generator Loss']
			fig_, ax_ = plt.subplots(2, 1, figsize=(12, 6*2))
			ax_[0].plot(D_loss.keys(), D_loss.values())
			ax_[1].plot(G_loss.keys(), G_loss.values())
			ax_[0].set_title(titles_[0])
			ax_[1].set_title(titles_[1])
			filename_ = 'loss_ep_%07d.png' % (i+1)
			plt.savefig(filename_)
			plt.close()
	titles = ['Discriminator Loss', 'Generator Loss']
	fig, ax = plt.subplots(2, 1, figsize=(12, 6*2))
	ax[0].plot(D_loss.keys(), D_loss.values())
	ax[1].plot(G_loss.keys(), G_loss.values())
	ax[0].set_title(titles[0])
	ax[1].set_title(titles[1])
	filename = 'loss_%07d.png' % (n_steps)
	plt.savefig(filename)
	plt.close()
	with open('d_loss_%07d.pkl' % (n_steps), 'wb') as f:
		pickle.dump(D_loss, f)
	with open('g_loss_%07d.pkl' % (n_steps), 'wb') as f:
		pickle.dump(G_loss, f)
##################################################################################################################

In [ ]:
##################################################################################################################
# load image data   
dataset = load_real_samples('*.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)

# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]

# define the models
d_model = define_discriminator(image_shape)
# g_model = define_generator(image_shape)
g_model = define_generator(image_shape)

# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)

# train model
train(d_model, g_model, gan_model, dataset)
##################################################################################################################

In [ ]:
# import pandas as pd
# from keras.models import load_model

# # load model
# model = load_model('model_0277200.h5')
# train_dt = load_real_samples('train_224_J.npz')
# test_dt = load_real_samples('test_224_J.npz')
# model.summary()
#loss, acc = model.evaluate(test_dt[0], test_dt[1], verbose=1)

# model_history = pd.DataFrame(model.history)
# model_history[2] = model.n_steps

# fig, ax = plt.subplots(1, figsize=(8,6))
# num_epochs = model_history.shape[0]

# ax.plot(np.arange(0, num_epochs), model_history["mae"], 
#         label="Training MAE")
# # ax.plot(np.arange(0, num_epochs), model_history["val_mae"], 
# #         label="Validation MAE")
# ax.legend()

# plt.tight_layout()
# plt.show()

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# # 캔버스 사이즈 적용
# plt.rcParams["figure.figsize"] = (12, 9)
# x = ...
# sns.distplot(x)
# plt.show()

In [ ]:
# print(len(train_dt[0]))

In [ ]:
# x = list(zip(train_dt[0], train_dt[1]))
# print(len(x))
# x1 = np.random.choice(27720, 2772)
# x2 = np.random.choice(27720, 2772)
# x3 = np.random.choice(27720, 2772)
# print(x1, x2, x3)

In [ ]:
# plt.rcParams["figure.figsize"] = (12, 9)
# sns.distplot(x1)
# sns.distplot(x2)
# sns.distplot(x3)
# plt.show()